---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [3]:
def date_sorter():
    
    # Your code here
    import re
    import dateutil.parser
    from datetime import datetime
    
    get_dates = pd.DataFrame(doc, columns=['text'])
    pattern=r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|(?:\d{1,2}[,]? )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*?[.,]? (?:\d{1,2}[,]? )?\d{4}|\d{1,2}[/]\d{4}|[12]\d{3}'
    get_dates['Date'] = get_dates['text'].apply(lambda x: re.findall(pattern,x))
    
    #remove [] bracket around the extract
    get_dates['Date'] = get_dates['Date'].apply(lambda x: x[0])
    
    #correct spelling mistakes in month
    #January
    index_jan = get_dates[get_dates.Date.str.contains(r'Janaury')].index.tolist()
    #index_jan 
    get_dates['Date'][298]=  re.sub(r'Janaury', 'January', get_dates['Date'][298])
    #December
    index_dec = get_dates[get_dates.Date.str.contains(r'Decemeber')].index.tolist()
    #index_dec
    get_dates['Date'][313]=  re.sub(r'Decemeber', 'December', get_dates['Date'][313])
    
    #for only two digits in the year add 19 in front
    list_years=list(get_dates['Date'])
    i=0

    pattern1=r'\d{1,2}[/]\d{1,2}[/]\d{2}'

    for item in list_years:
        if(re.match(pattern1, item)):
            if len(item.split('/')[2])==2:
                list_years[i] = item.split('/')[0]+'/'+item.split('/')[1]+'/'+'19'+item.split('/')[2]

        i=i+1
    
    # replace formated dates back to dates   
    get_dates['Date']=list_years
    
    #conver strings to dates
    list_dates=list(get_dates['Date'])
    i=0

    pattern1=r'[12]\d{3}'
    month_day='January 1'
    pattern2=r'\d{1,2}[/]\d{4}'
    pattern3=r'[A-Z][a-z]+[-,]? \d{4}'

    for item in list_dates:

        if(re.match(pattern1, item)):
            list_dates[i] = month_day+' '+list_dates[i]
        elif(re.match(pattern2, item)):
            list_dates[i] = item.split('/')[0]+'/01/'+item.split('/')[1]
        elif(re.match(pattern3, item)):
            list_dates[i] = item.split(' ')[0]+'/01/'+item.split(' ')[1]

        list_dates[i]=pd.to_datetime(list_dates[i], format="%m/%d/%Y", infer_datetime_format=True)

        i=i+1

    # replace formated dates back to dates   
    get_dates['Date']=list_dates
    
    get_dates.drop('text', axis=1,inplace=True)

    # Sort the list in ascending order of dates
    result_series = pd.Series(get_dates['Date'].sort_values().index)    

    return result_series# Your answer here